# NOTE: Main Changes of Original Model
## Try Different Parameters in Each Model for Multiple Times (Detailed Outputs in the Excel Files - Post On BB)
## 2 Add a XGBoost At the End of This File


# Modeling

In [1]:
from datetime import datetime
start_time = datetime.now()

# Libraries to load
# !pip install lightgbm

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import numpy as np
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:02.906858


In [2]:
%%time
vars_total = pd.read_csv('vars_short.csv')

CPU times: user 2min 35s, sys: 55.2 s, total: 3min 30s
Wall time: 3min 38s


In [36]:
# use this to cap variables. For some problems it helps
Clip = 10

In [37]:
vars.rename(columns={'fraud_label':'Fraud'},inplace=True)

In [38]:
vars['Fraud'].sum()

14393

In [39]:
vars.head(10)

,record,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,ssn_age_level_count_0_by_30,ssn_age_level_count_0_by_14,name_dob_count_30,ssn_name_count_30,...,ssn_age_level_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,name_dob_count_0_by_30,ssn_count_0_by_14,ssn_name_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_name_dob_count_7,ssn_name_count_0_by_14,Fraud
0,1,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
1,2,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,1
2,3,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
3,4,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
4,5,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
5,6,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
6,7,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
7,8,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
8,9,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0
9,10,1,1,1,1,1,30.0,14.0,1,1,...,1,1,1,30.0,14.0,14.0,14.0,1,14.0,0


In [40]:
vars.shape

(1000000, 27)

In [41]:
vars.describe()

,record,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,ssn_age_level_count_0_by_30,ssn_age_level_count_0_by_14,name_dob_count_30,ssn_name_count_30,...,ssn_age_level_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,name_dob_count_0_by_30,ssn_count_0_by_14,ssn_name_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_name_dob_count_7,ssn_name_count_0_by_14,Fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,500000.500000,1.064624,1.048903,1.449296,1.046097,1.048207,29.545788,13.885564,1.046295,1.048778,...,1.033714,1.032193,1.033601,29.584207,13.882687,13.895295,13.885818,1.025098,13.886118,0.014393
std,288675.278933,0.633831,0.499029,0.861622,0.496315,0.507682,2.653148,0.926658,0.496914,0.498892,...,0.473324,0.471479,0.472915,2.546141,0.940186,0.889286,0.924867,0.451121,0.923737,0.119104
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.363636,0.636364,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.363636,0.636364,0.636364,0.636364,1.000000,0.636364,0.000000
25%,250000.750000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,30.000000,14.000000,14.000000,14.000000,1.000000,14.000000,0.000000
50%,500000.500000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,30.000000,14.000000,14.000000,14.000000,1.000000,14.000000,0.000000
75%,750000.250000,1.000000,1.000000,2.000000,1.000000,1.000000,30.000000,14.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,30.000000,14.000000,14.000000,14.000000,1.000000,14.000000,0.000000
max,1000000.000000,30.000000,34.000000,32.000000,34.000000,30.000000,30.000000,14.000000,34.000000,34.000000,...,34.000000,34.000000,34.000000,30.000000,14.000000,14.000000,14.000000,34.000000,14.000000,1.000000


In [42]:
record_save = vars['record']
Y_save = pd.DataFrame(vars.loc[:,'Fraud'])
Y_save.head()

,Fraud
0,0
1,1
2,0
3,0
4,0


## Scale and truncate field values

In [43]:
X_no_scaling = vars.drop(columns = ['record','Fraud'])
X_no_scaling.describe()

,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,ssn_age_level_count_0_by_30,ssn_age_level_count_0_by_14,name_dob_count_30,ssn_name_count_30,ssn_dob_count_14,...,name_dob_count_14,ssn_age_level_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,name_dob_count_0_by_30,ssn_count_0_by_14,ssn_name_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_name_dob_count_7,ssn_name_count_0_by_14
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.00000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,1.064624,1.048903,1.449296,1.046097,1.048207,29.545788,13.885564,1.046295,1.048778,1.03226,...,1.032363,1.033714,1.032193,1.033601,29.584207,13.882687,13.895295,13.885818,1.025098,13.886118
std,0.633831,0.499029,0.861622,0.496315,0.507682,2.653148,0.926658,0.496914,0.498892,0.47155,...,0.471763,0.473324,0.471479,0.472915,2.546141,0.940186,0.889286,0.924867,0.451121,0.923737
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.363636,0.636364,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.363636,0.636364,0.636364,0.636364,1.000000,0.636364
25%,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,14.000000,14.000000,1.000000,14.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,14.000000,14.000000,1.000000,14.000000
75%,1.000000,1.000000,2.000000,1.000000,1.000000,30.000000,14.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,30.000000,14.000000,14.000000,14.000000,1.000000,14.000000
max,30.000000,34.000000,32.000000,34.000000,30.000000,30.000000,14.000000,34.000000,34.000000,34.00000,...,34.000000,34.000000,34.000000,34.000000,30.000000,14.000000,14.000000,14.000000,34.000000,14.000000


In [44]:
X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()

In [45]:
# push in any outlier values
cols = X.columns
X.loc[:,cols] = X[cols].clip(upper=Clip)
X.loc[:,cols] = X[cols].clip(lower=-1*Clip)
# X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
X.describe()

,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,ssn_age_level_count_0_by_30,ssn_age_level_count_0_by_14,name_dob_count_30,ssn_name_count_30,ssn_dob_count_14,...,name_dob_count_14,ssn_age_level_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,name_dob_count_0_by_30,ssn_count_0_by_14,ssn_name_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_name_dob_count_7,ssn_name_count_0_by_14
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,-0.018176,-0.017457,-0.005814,-0.017649,-0.018020,0.000035,0.002577,-0.017617,-0.017467,-0.018502,...,-0.018486,-0.018362,-0.018507,-0.018393,0.000159,0.002583,0.003333,0.002572,-0.018700,0.002588
std,0.631721,0.579481,0.910935,0.571302,0.576299,0.999643,0.970439,0.572884,0.579076,0.519100,...,0.519888,0.525781,0.518828,0.524293,0.998362,0.970586,0.961009,0.970455,0.483844,0.970243
min,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955,-10.000000,-10.000000,-0.093165,-0.097773,-0.068413,...,-0.068600,-0.071228,-0.068281,-0.071051,-10.000000,-10.000000,-10.000000,-10.000000,-0.055635,-10.000000
25%,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955,0.171197,0.123493,-0.093165,-0.097773,-0.068413,...,-0.068600,-0.071228,-0.068281,-0.071051,0.163303,0.124777,0.117741,0.123458,-0.055635,0.123284
50%,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955,0.171197,0.123493,-0.093165,-0.097773,-0.068413,...,-0.068600,-0.071228,-0.068281,-0.071051,0.163303,0.124777,0.117741,0.123458,-0.055635,0.123284
75%,-0.101958,-0.097996,0.639148,-0.092879,-0.094955,0.171197,0.123493,-0.093165,-0.097773,-0.068413,...,-0.068600,-0.071228,-0.068281,-0.071051,0.163303,0.124777,0.117741,0.123458,-0.055635,0.123284
max,10.000000,10.000000,10.000000,10.000000,10.000000,0.171197,0.123493,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,0.163303,0.124777,0.117741,0.123458,10.000000,0.123284


In [46]:
# separate data into modeling (traintest) and out of time
X_trntst = X[0:833507]
Y_trntst = Y_save[0:833507]
X_oot = X[833507:]
Y_oot = Y_save[833507:]

In [47]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

# ALL THE CODES ABOVE THIS CELL SHOULD BE RUN

In [51]:
%%time
# Logistic regression
log = pd.DataFrame(columns=['num_vars','penalty','solver','c','train','test','oot'])
number_var = []
penalty_num = []
solver_type = []
c_value = []
train_value = []
test_value = []
oot_value = []
for num_vars in [i for i in range(6,21) if i%2==0]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for penalty in ['l1','l2']:
        for solver in ['lbfgs','saga']:
            for c in [0.1,0.3,0.6,1,3]:
                try:
                    for niter in range(nitermax):    
                        X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                        model = LogisticRegression(penalty=penalty,solver=solver,C=c)

                        X_oot = X_oot_orig.copy()
                        X_trn_save = X_trn.copy()
                        Y_trn_save = Y_trn.copy()

                        model.fit(X_trn, Y_trn.values.ravel())   
                        predictions = model.predict_proba(X_trn_save)[:,1]
                        X_trn['predicted'] = predictions
                        X_trn['Fraud'] = Y_trn_save['Fraud']
                        topRows = int(round(X_trn.shape[0]*0.03))
                        temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_tst)[:,1]
                        X_tst['predicted']=predictions
                        X_tst['Fraud'] = Y_tst['Fraud']
                        topRows = int(round(X_tst.shape[0]*0.03))
                        temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_oot_new)[:,1]
                        X_oot['predicted']=predictions
                        X_oot['Fraud'] = Y_oot['Fraud']
                        topRows = int(round(X_oot.shape[0]*0.03))
                        temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                    number_var.append(num_vars)
                    penalty_num.append(penalty)
                    solver_type.append(solver)
                    c_value.append(c)
                    train_value.append(round(FDR3.mean()[0]*100,2))
                    test_value.append(round(FDR3.mean()[1]*100,2))
                    oot_value.append(round(FDR3.mean()[2]*100,2))
                    print("finished!")
                except:
                    print('bad_model')

good
bad_model
bad_model
bad_model
bad_model
finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!


...

finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
CPU times: user 12min 14s, sys: 5.21 s, total: 12min 19s
Wall time: 9min 53s


In [52]:
logm = pd.DataFrame(data={'num_vars':number_var,'penalty':penalty_num,'solver':solver_type,'c':c_value,'train':train_value,'test':test_value,'oot':oot_value})

In [53]:
logm.to_excel("ligstic_MultipleVarsTrial.xlsx")

## Models. You can comment out any of these cells and just explore one model type. You can also just rerun that single cell multiple times as you explore different model hyperparameters.

In [54]:
%%time
# Single DT
number_var = []
criterion_type = []
splitter_type = []
max_depth_num= []
min_samples_leaf_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [i for i in range(6,21) if i%2==0]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for criterion in ['gini','entropy']:
        for splitter in ['best']:
            for max_depth in [5,10,15,20]:
                for min_samples_leaf in [15,20,25,30,40]:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = DecisionTreeClassifier(criterion=criterion,splitter=splitter,max_depth=max_depth,min_samples_leaf=min_samples_leaf)

                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                        number_var.append(num_vars)
                        criterion_type.append(criterion)
                        splitter_type.append(splitter)
                        max_depth_num.append(max_depth)
                        min_samples_leaf_num.append(min_samples_leaf)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model!")

good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
CPU times: user 1min 57s, sys: 16.8 s, total: 2min 14s
Wall time: 2min 14s


In [55]:
dt = pd.DataFrame(data={'num_vars':number_var,'criterion':criterion_type,'splitter':splitter_type,'max_depth':max_depth_num,'min_samples_leaf':min_samples_leaf_num,'train':train_value,'test':test_value,'oot':oot_value})

In [56]:
dt.to_excel("decision_MultipleVarsTrial.xlsx")

In [23]:
%%time
# RF
number_var = []
criterion_type = []
n_estimators_num = []
max_depth_num= []
max_features_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for n_estimators in [10,50,100,150]:
        for max_depth in [10,20,30]:
            for cirterion in ['gini','entropy']:
                for max_features in ["auto","log2"]:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,criterion=criterion,max_features=max_features)
                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                            
                        number_var.append(num_vars)
                        criterion_type.append(criterion)
                        n_estimators_num.append(n_estimators)
                        max_depth_num.append(max_depth)
                        max_features_num.append(max_features)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model!")

good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
CPU times: user 1h 29min 16s, 

In [24]:
rf = pd.DataFrame(data={'num_vars':number_var,'criterion':criterion_type,'n_estimators':n_estimators_num,'max_depth':max_depth_num,'max_features':max_features_num,'train':train_value,'test':test_value,'oot':oot_value})

In [25]:
rf.to_excel('rf.xlsx')

In [26]:
%%time
# BT
number_var = []
learning_rate_num = []
n_estimators_num = []
max_depth_num= []
subsample_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [15,20]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for n_estimators in [100,200,500,1000]:
        for max_depth in [3,4,5]:
            for learning_rate in [0.001,0.01,0.02,0.05,0.1]:
                for subsample in [0.7,0.8,1]:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = lgb.LGBMClassifier(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate,subsample=subsample)

                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                            
                        number_var.append(num_vars)
                        learning_rate_num.append(learning_rate)
                        n_estimators_num.append(n_estimators)
                        max_depth_num.append(max_depth)
                        subsample_num.append(subsample)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model")

good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finis

In [27]:
bt = pd.DataFrame(data={'num_vars':number_var,'learning_rate':learning_rate_num,'n_estimators':n_estimators_num,'max_depth':max_depth_num,'subsample':subsample_num,'train':train_value,'test':test_value,'oot':oot_value})

In [28]:
bt.to_excel("bt.xlsx")

In [57]:
%%time
# NN
number_var = []
hidden_layer_size_num = []
max_iter_num = []
learning_rate_type = []
train_value = []
test_value = []
oot_value = []
for num_vars in [6,8,12,16]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for hidden_layer_sizes in [20,(20,10),(10,5)]:
        for max_iter in [20,40,60,100]:
            for learning_rate in ['invscaling','constant']:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,max_iter=max_iter,learning_rate=learning_rate)

                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                            
                        number_var.append(num_vars)
                        hidden_layer_size_num.append(hidden_layer_sizes)
                        max_iter_num.append(max_iter)
                        learning_rate_type.append(learning_rate)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model")

good
finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
CPU times: user 22min 24s, sys: 11.8 s, total: 22min 36s
Wall time: 19min 41s


In [58]:
nn = pd.DataFrame(data={'num_vars':number_var,'hidden_layer_size':hidden_layer_size_num,'max_iter':max_iter_num,'learning_rate':learning_rate_type,'train':train_value,'test':test_value,'oot':oot_value})

In [59]:
nn.to_excel("nn_MultipleVarsTrial.xlsx")

In [60]:
%%time
# XGBoost
from xgboost import XGBClassifier
number_var = []
learning_rate_num = []
n_estimators_num = []
max_depth_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [8,10,12,16]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for n_estimators in [100,200,500,800,1000]:
        for learning_rate in [0.05,0.1,0.3,0.5]:
            for max_depth in [3,4,5]:
                try:
                    for niter in range(nitermax):    
                        X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                        model = XGBClassifier(seed=47)

                        X_oot = X_oot_orig.copy()
                        X_trn_save = X_trn.copy()
                        Y_trn_save = Y_trn.copy()

                        model.fit(X_trn, Y_trn.values.ravel())   

                        predictions = model.predict_proba(X_trn_save)[:,1]
                        X_trn['predicted'] = predictions
                        X_trn['Fraud'] = Y_trn_save['Fraud']
                        topRows = int(round(X_trn.shape[0]*0.03))
                        temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_tst)[:,1]
                        X_tst['predicted']=predictions
                        X_tst['Fraud'] = Y_tst['Fraud']
                        topRows = int(round(X_tst.shape[0]*0.03))
                        temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_oot_new)[:,1]
                        X_oot['predicted']=predictions
                        X_oot['Fraud'] = Y_oot['Fraud']
                        topRows = int(round(X_oot.shape[0]*0.03))
                        temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])

                    number_var.append(num_vars)
                    learning_rate_num.append(learning_rate)
                    n_estimators_num.append(n_estimators)
                    max_depth_num.append(max_depth)
                    train_value.append(round(FDR3.mean()[0]*100,2))
                    test_value.append(round(FDR3.mean()[1]*100,2))
                    oot_value.append(round(FDR3.mean()[2]*100,2))
                    print("finished!")
                except:
                    print("bad model")

good


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:40:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:40:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:40:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[12:40:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

[12:43:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[12:44:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:44:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:44:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

[12:47:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:47:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[12:47:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:47:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

finished!
[12:50:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:50:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:50:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[12:51:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation me

[12:54:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[12:54:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:54:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:54:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

[12:57:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[12:57:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:57:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

finished!
[13:00:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:00:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:01:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[13:01:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation me

In [61]:
xgboost = pd.DataFrame(data={'num_vars':number_var,'n_estimators':n_estimators_num,'learning_rate':learning_rate_num,'max_depth':max_depth_num,'train':train_value,'test':test_value,'oot':oot_value})

In [62]:
xgboost.to_excel('xgboost_MultipleVarsTrial.xlsx')

In [19]:
print('duration: ', datetime.now() - start_time)

duration:  0:03:48.900114
